## Gaurav Kumarr
#### **Content Based Movie Recommendation System**

In [ ]:
#install kaggle
! pip install kaggle

In [ ]:
#making a directory for kaggle files,Remember directory can't named anything else than kaggle
! mkdir ~/.kaggle

###### ***Download kaggle.json file from your kaggle account , go to setting and create a new token***

In [ ]:
#copy your kaggle.json file to kaggle directory
! cp kaggle.json ~/.kaggle

###### ***Download rotten tomato dataset from kaggle data by copying API command and pasting it in colab or you can directly download zip file***

In [ ]:
! kaggle datasets download -d andrezaza/clapper-massive-rotten-tomatoes-movies-and-reviews

In [ ]:
!unzip /content/clapper-massive-rotten-tomatoes-movies-and-reviews.zip

###### *Importing all the necessary packages and libraries*

In [5]:
#!!important!!
import pandas as pd
import numpy as np
import nltk
import Levenshtein
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
# import stemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder,OneHotEncoder,MinMaxScaler,normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from tkinter import messagebox
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import tkinter as tk
from Levenshtein import distance
# from plotly.offline import init_notebook_mode,iplot
import matplotlib as mpl
import matplotlib.pyplot as plt




import re
import seaborn as sns

###### **Read extracted csv file using panda and read_csv function ,copy the file path and if you are using windows then use \\\ instead of /**

In [6]:
#!!important!!
df_movie = pd.read_csv("C:\\Users\\gaura\\Desktop\\Rotten_tomato\\movies.csv")
df_review = pd.read_csv("C:\\Users\\gaura\\Desktop\\Rotten_tomato\\movie_reviews.csv")

In [7]:
#Movie data frame from csv file
df_movie

,id,title,audienceScore,tomatoMeter,rating,ratingContents,releaseDateTheaters,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director,writer,boxOffice,distributor,soundMix
0,space-zombie-bingo,Space Zombie Bingo!,50.0,NaN,NaN,NaN,NaN,2018-08-25,75.0,"Comedy, Horror, Sci-fi",English,George Ormrod,"George Ormrod,John Sabotta",NaN,NaN,NaN
1,the_green_grass,The Green Grass,NaN,NaN,NaN,NaN,NaN,2020-02-11,114.0,Drama,English,Tiffany Edwards,Tiffany Edwards,NaN,NaN,NaN
2,love_lies,"Love, Lies",43.0,NaN,NaN,NaN,NaN,NaN,120.0,Drama,Korean,"Park Heung-Sik,Heung-Sik Park","Ha Young-Joon,Jeon Yun-su,Song Hye-jin",NaN,NaN,NaN
3,the_sore_losers_1997,Sore Losers,60.0,NaN,NaN,NaN,NaN,2020-10-23,90.0,"Action, Mystery & thriller",English,John Michael McCarthy,John Michael McCarthy,NaN,NaN,NaN
4,dinosaur_island_2002,Dinosaur Island,70.0,NaN,NaN,NaN,NaN,2017-03-27,80.0,"Fantasy, Adventure, Animation",English,Will Meugniot,John Loy,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143253,nadia_the_secret_of_blue_water_the_motion_pict...,Nadia: The Secret of Blue Water: The Motion Pi...,14.0,NaN,NaN,NaN,2002-08-27,NaN,90.0,"Action, Adventure, Anime",Japanese,Sho Aono,Kaoru Umeno,NaN,ADV Films,NaN
143254,everyone_i_knew_and_loved,Everyone I Knew and Loved,NaN,NaN,NaN,NaN,NaN,NaN,99.0,Drama,English,Andrew Behringer,Erika Heidewald,NaN,NaN,NaN
143255,the-human-body,The Human Body,71.0,89.0,NaN,NaN,NaN,NaN,43.0,Documentary,English,Peter Georgi,Richard Dale,NaN,NaN,NaN
143256,flying_fists,Flying Fists,NaN,NaN,NaN,NaN,NaN,2006-11-21,63.0,Drama,English,Robert F. Hill,"Robert F. Hill,Basil Dickey",NaN,NaN,NaN


In [8]:
#review data frame from csv file
df_review

,id,reviewId,creationDate,criticName,isTopCritic,originalScore,reviewState,publicatioName,reviewText,scoreSentiment,reviewUrl
0,beavers,1145982,2003-05-23,Ivan M. Lincoln,False,3.5/4,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,http://www.deseretnews.com/article/700003233/B...
1,blood_mask,1636744,2007-06-02,The Foywonder,False,1/5,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,http://www.dreadcentral.com/index.php?name=Rev...
2,city_hunter_shinjuku_private_eyes,2590987,2019-05-28,Reuben Baron,False,NaN,fresh,CBR,The choreography is so precise and lifelike at...,POSITIVE,https://www.cbr.com/city-hunter-shinjuku-priva...
3,city_hunter_shinjuku_private_eyes,2558908,2019-02-14,Matt Schley,False,2.5/5,rotten,Japan Times,The film's out-of-touch attempts at humor may ...,NEGATIVE,https://www.japantimes.co.jp/culture/2019/02/0...
4,dangerous_men_2015,2504681,2018-08-29,Pat Padua,False,NaN,fresh,DCist,Its clumsy determination is endearing and some...,POSITIVE,http://dcist.com/2015/11/out_of_frame_dangerou...
...,...,...,...,...,...,...,...,...,...,...,...
1444958,thor_love_and_thunder,102706151,2022-07-05,Christie Cronan,False,7/10,fresh,Raising Whasians,Solid but not totally sold&#44; Thor&#58; Ragn...,POSITIVE,https://raisingwhasians.com/thor-love-and-thun...
1444959,thor_love_and_thunder,102706150,2022-07-05,Ian Sandwell,False,4/5,fresh,Digital Spy,Thor&#58; Love and Thunder is the most enterta...,POSITIVE,https://www.digitalspy.com/movies/a40496050/th...
1444960,thor_love_and_thunder,102706149,2022-07-05,Lauren LaMagna,False,8/10,fresh,Next Best Picture,&quot;Thor&#58; Love and Thunder&quot; is a st...,POSITIVE,https://www.nextbestpicture.com/thor-love-and-...
1444961,thor_love_and_thunder,102706148,2022-07-05,Jake Cole,True,1/4,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,https://www.slantmagazine.com/film/thor-love-a...


In [9]:
#number of rows in a dataframe
row_movie=len(df_movie)
row_review=len(df_review)

In [ ]:
#Description of movie data frame
df_movie.describe()

In [ ]:
#Description of review data frame
df_review.describe()

In [10]:
#null value column description in movie data frame
df_movie.isnull().sum()

id                           0
title                      367
audienceScore            70010
tomatoMeter             109381
rating                  129267
ratingContents          129267
releaseDateTheaters     112485
releaseDateStreaming     63838
runtimeMinutes           13827
genre                    11083
originalLanguage         13858
director                  4217
writer                   53142
boxOffice               128515
distributor             120257
soundMix                127341
dtype: int64

In [11]:
#null value column description in review data frame
df_review.isnull().sum()

id                     0
reviewId               0
creationDate           0
criticName             0
isTopCritic            0
originalScore     435218
reviewState            0
publicatioName         0
reviewText         69225
scoreSentiment         0
reviewUrl         210925
dtype: int64

###### ***visualisation and understanding of data***

In [ ]:
df_movie = df_movie[df_movie.releaseDateTheaters.notnull()]
df_movie['releaseDateTheaters'] = pd.to_datetime(df_movie['releaseDateTheaters'])
df_movie['movie_year'] = df_movie['releaseDateTheaters'].apply(lambda x: x.year)

sns.set(style="white")

plt.figure(figsize=(15,10))
plt.title('Movies by the year', size=20)
sns.distplot(df_movie.movie_year, kde=False)
plt.ylabel('Number of movies', size=15)
plt.xlabel('Year of release',size=15)
plt.axis([1920, 2019, 0, 1750])
plt.xticks(np.arange(1850, 2018, step=5),rotation=45, ha='right')
plt.show()

In [ ]:
df_review = df_review[df_review.creationDate.notnull()]
df_review['creationDate'] = pd.to_datetime(df_review['creationDate'])
df_review['review_year'] = df_review['creationDate'].apply(lambda x: x.year)
df_review = df_review[df_review.review_year.astype(int) >= 2000]

plt.figure(figsize=(15,10))
plt.title('Reviews by the year', size=20)
sns.distplot(df_review.review_year, bins=20, kde=False)
plt.ylabel('Number of critic reviews', size=15)
plt.xlabel('Year of review posted',size=15)
plt.axis([2000, 2019, 0, 75000])
plt.xticks(np.arange(2000, 2022, step=1),rotation=45, ha='right')
plt.show()

In [ ]:
df_movie = df_movie[(df_movie.tomatoMeter.notnull()) &
                      (df_movie.audienceScore.notnull())]
sns.jointplot(x=df_movie['movie_year'], y=df_movie['tomatoMeter'],
              kind="kde").fig.set_size_inches(15,15)

In [ ]:
sns.jointplot(x=movies_df['movie_year'], y=movies_df['audienceScore'],
              kind="kde").fig.set_size_inches(15,15)

In [ ]:
a = plt.cm.cool

plt.figure(figsize=(15,10))
count = movies_df['distributor'].value_counts()[:10]
sns.barplot(count.values, count.index, palette=[a(0.1),a(0.2),a(0.3),a(0.4),a(0.5),a(0.6),a(0.7),a(0.8),a(0.9),a(0.99)])
for i, v in enumerate(count.values):
    plt.text(0.8,i,v,color='k',fontsize=14)
plt.xlabel('Count', fontsize=12)
plt.ylabel('Studio name', fontsize=12)
plt.title("Distribution of Studio names", fontsize=16)

In [ ]:
#fequency of movie genre
movies_df['first_genre'] = movies_df['genre'].str.split(',').str[0]

a = plt.cm.cool

plt.figure(figsize=(15,10))
count = movies_df['first_genre'].value_counts()[:7]
sns.barplot(count.values, count.index, palette=[a(0.1),a(0.2),a(0.3),a(0.4),a(0.5),a(0.6),a(0.7)])
for i, v in enumerate(count.values):
    plt.text(0.8,i,v,color='k',fontsize=14)
plt.xlabel('Count', fontsize=12)
plt.ylabel('Genre name', fontsize=12)
plt.title("Distribution of Genres", fontsize=16)

In [27]:
df_movie['director']

0                         george ormrod
2         park heung-sik,heung-sik park
3                 john michael mccarthy
4                         will meugniot
5                     baltasar kormákur
                      ...              
143249                       neil jones
143250                    josh schwartz
143252                  manish vatsalya
143253                         sho aono
143255                     peter georgi
Name: director, Length: 83335, dtype: object

In [29]:
df_movie['originalLanguage'].value_counts()

originalLanguage
English            58900
Spanish             2355
French (Canada)     2061
Hindi               1935
Chinese             1868
                   ...  
smi                    1
Aramaic                1
Tajik                  1
Malagasy               1
shg                    1
Name: count, Length: 111, dtype: int64

In [ ]:
non_null_languages = df_movie['originalLanguage'].dropna()

# Count occurrences of 'English' in the Language column
english_count = non_null_languages[non_null_languages == 'English'].value_counts()

# Print the count
english_count

originalLanguage
English    57684
Name: count, dtype: int64

###### *******Selecting important columns from data frame and removing irrelevant columns from movie data frame********

In [12]:
#!!important!! 
df_movie=df_movie[['id','title','audienceScore', 'tomatoMeter', 'genre','originalLanguage','director' ]]

###### ***replacing null values in title column of movie dataframe by id column as there is no null value in id, We are cleaning punctuation adn alphanumeric characters first using a function**

In [13]:
#!!important!!
def remove_alphanumeric(value):
    return re.sub('[^A-Za-z]+', '', value)

# Replace null values in Title column with modified MovieID values
df_movie['title'] = df_movie['title'].fillna(df_movie['id'].apply(remove_alphanumeric))

C:\Users\gaura\AppData\Local\Temp\ipykernel_8228\2387071881.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movie['title'] = df_movie['title'].fillna(df_movie['id'].apply(remove_alphanumeric))


In [14]:
#!!important!!
# Drop rows where genre or director is null
df_movie =df_movie.dropna(subset=['genre', 'director'])


In [15]:
#!!important!!
#delete duplicate rows from movie data frame
df_movie = df_movie.drop_duplicates()

##### ***removing all rows that has no rating at all***

In [16]:
#!!important!!
# Filter rows where audiencescore and originalscore  column is null
null_language_df = df_movie[(df_movie['tomatoMeter'].isnull()) & (df_movie['audienceScore'].isnull())]

merged_df = null_language_df.merge(df_review, on='id', how='left', indicator=True)

# Filter rows where the movie has no data in the reviews DataFrame
no_review_df = merged_df[merged_df['_merge'] == 'left_only']



In [17]:
#!!important!!
# removing all rows that has no rating at all
mask = df_movie['id'].isin(no_review_df['id'])

# Remove the rows with True values in the mask from the original DataFrame
df_movie = df_movie[~mask]

In [18]:
#!!important!!
# replacing null values from originalLanguage with 'unknown'
df_movie['originalLanguage'] = df_movie['originalLanguage'].fillna('Unknown')

###### ***Calculating rating based on conditions provided and combining all rating together***

In [19]:
#!!important!!
# Identify movies with null Tomatometer values in the movie DataFrame
missing_tomatometer_indices = df_movie['tomatoMeter'].isnull()

if missing_tomatometer_indices.any():
    # Merge movie and review DataFrames based on a common column, e.g., 'movie_id'
    merged_df = df_movie.merge(df_review, on='id', how='left')
    
    # Calculate the rating based on the conditions provided
    merged_df['rating'] = np.where(
        merged_df['isTopCritic'] == 'TRUE',
        np.where(merged_df['scoreSentiment'] == 'POSITIVE', 100, 10),
        np.where(merged_df['scoreSentiment'] == 'POSITIVE', 80, 40)
    )
    
    # Group by movie and calculate the average rating
    movie_avg_rating = merged_df.groupby('id')['rating'].mean()
    
    # Update the movie DataFrame with the calculated average ratings
    df_movie.loc[missing_tomatometer_indices, 'tomatoMeter'] = df_movie.loc[missing_tomatometer_indices, 'id'].map(movie_avg_rating)

In [20]:
#!!important!!
missing_audience_score_indices = df_movie['audienceScore'].isnull()

if missing_audience_score_indices.any():
    # Substitute the value of "audienceScore" with the value of "tomatometer"
    df_movie .loc[missing_audience_score_indices, 'audienceScore'] = df_movie.loc[missing_audience_score_indices, 'tomatoMeter']

# Create a new column "averageScore" with the average value of "audienceScore" and "tomatometer"
df_movie['averageScore'] = (df_movie['audienceScore'] + df_movie['tomatoMeter']) / 2

In [16]:
df_review

,id,reviewId,creationDate,criticName,isTopCritic,originalScore,reviewState,publicatioName,reviewText,scoreSentiment,reviewUrl
0,beavers,1145982,2003-05-23,Ivan M. Lincoln,False,3.5/4,fresh,Deseret News (Salt Lake City),Timed to be just long enough for most youngste...,POSITIVE,http://www.deseretnews.com/article/700003233/B...
1,blood_mask,1636744,2007-06-02,The Foywonder,False,1/5,rotten,Dread Central,It doesn't matter if a movie costs 300 million...,NEGATIVE,http://www.dreadcentral.com/index.php?name=Rev...
2,city_hunter_shinjuku_private_eyes,2590987,2019-05-28,Reuben Baron,False,NaN,fresh,CBR,The choreography is so precise and lifelike at...,POSITIVE,https://www.cbr.com/city-hunter-shinjuku-priva...
3,city_hunter_shinjuku_private_eyes,2558908,2019-02-14,Matt Schley,False,2.5/5,rotten,Japan Times,The film's out-of-touch attempts at humor may ...,NEGATIVE,https://www.japantimes.co.jp/culture/2019/02/0...
4,dangerous_men_2015,2504681,2018-08-29,Pat Padua,False,NaN,fresh,DCist,Its clumsy determination is endearing and some...,POSITIVE,http://dcist.com/2015/11/out_of_frame_dangerou...
...,...,...,...,...,...,...,...,...,...,...,...
1444958,thor_love_and_thunder,102706151,2022-07-05,Christie Cronan,False,7/10,fresh,Raising Whasians,Solid but not totally sold&#44; Thor&#58; Ragn...,POSITIVE,https://raisingwhasians.com/thor-love-and-thun...
1444959,thor_love_and_thunder,102706150,2022-07-05,Ian Sandwell,False,4/5,fresh,Digital Spy,Thor&#58; Love and Thunder is the most enterta...,POSITIVE,https://www.digitalspy.com/movies/a40496050/th...
1444960,thor_love_and_thunder,102706149,2022-07-05,Lauren LaMagna,False,8/10,fresh,Next Best Picture,&quot;Thor&#58; Love and Thunder&quot; is a st...,POSITIVE,https://www.nextbestpicture.com/thor-love-and-...
1444961,thor_love_and_thunder,102706148,2022-07-05,Jake Cole,True,1/4,rotten,Slant Magazine,Across Taika Waititi&#8217;s film&#44; a war a...,NEGATIVE,https://www.slantmagazine.com/film/thor-love-a...


In [15]:
df_review.isnull().sum()

id                     0
reviewId               0
creationDate           0
criticName             0
isTopCritic            0
originalScore     431220
reviewState            0
publicatioName         0
reviewText             0
scoreSentiment         0
reviewUrl         162592
dtype: int64

In [ ]:
df_review=df_review[['id','isTopCritic','reviewState','reviewText']]

In [ ]:
df_review['originalScore'].describe()

count     1009745
unique       1729
top           3/5
freq       116711
Name: originalScore, dtype: object

In [ ]:
review_column = df_review['reviewText'].head(20)

# Display the full review column
  # Show full content of each cell
print(review_column)

0                                                                                          Timed to be just long enough for most youngsters' brief attention spans -- and it's packed with plenty of interesting activity, both on land and under the water.
1                                                                                       It doesn't matter if a movie costs 300 million or only 300 dollars; good is good and bad is bad, and Bloodmask: The Possession of Nicole Lameroux is just plain bad.
2     The choreography is so precise and lifelike at points one might wonder whether the movie was rotoscoped, but no live-action reference footage was used. The quality is due to the skill of the animators and Kodama's love for professional wrestling.
3                                                                                                                            The film's out-of-touch attempts at humor may find them hunting for the reason the franchise was so popular in the f

In [21]:
#!!important!!
#dropping duplicates
df_review = df_review.drop_duplicates()

In [22]:
#!!important!!
#dropping all rows having null values
df_review = df_review.dropna(subset=['reviewText'])

###### ***Step 2: Extracting movie reviews from the dataset and grouping all reviews for each movie***

###### ***Merging the movie dataset with the review dataset based on movie id***

In [23]:
#!!important!!
# Step 2: Concatenate all reviews for each movie


reviews_concat = df_review.groupby('id')['reviewText'].apply(lambda x: ' '.join(x)).reset_index()
movie_reviews = pd.merge(df_movie, reviews_concat, on='id', how='left')




###### ***Text Preprocessing the data for text analysis and extracting features from it, we are using stemming fro getting root words for each word and and we are using tdfidf vectorizer for extracting features,it is a term frequency inverse document frequency vectorizer which is used for text classification and it is useful here to understand the importance of each word, we are using english stop words to remove useless words we have modified tdfidf vectorizer and added stemmer**

In [24]:
#!!important!!
from sklearn.feature_extraction.text import TfidfVectorizer
import Stemmer

# English stemmer from pyStemmer
stemmer = Stemmer.Stemmer('en')

analyzer = TfidfVectorizer().build_analyzer()

# Override TfidfVectorizer
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: stemmer.stemWords(analyzer(doc))

# Create a new StemmedTfidfVectorizer
vectorizer = StemmedTfidfVectorizer(stop_words='english',max_features=10000)
matrix = vectorizer.fit_transform(movie_reviews['reviewText'].fillna(''))

In [17]:
print(movie_reviews['originalLanguage'].head(20))

0     english
1      korean
2     english
3     english
4     english
5     spanish
6     english
7     english
8     english
9     english
10    english
11    tagalog
12      hindi
13    english
14      hindi
15    english
16    english
17    english
18    english
19    english
Name: originalLanguage, dtype: object


###### ****Text preprocessing for genre, director, original language and averageScore columns and extracting features from it, we are using multilabelbinarizer for genre and labelencoder for director and onehotencoder for original language and minmaxscaler for averageScore****

In [25]:
#!!important!!
# Text preprocessing for genre
# Remove all non-alphanumeric characters from genre column

movie_reviews['genre'] = movie_reviews['genre'].str.lower().str.replace('[^\w\s]', '')
movie_reviews['genre'] = movie_reviews['genre'].str.split(',').apply(lambda x: [genre.strip() for genre in x])

# Text preprocessing for director
movie_reviews['director'] = movie_reviews['director'].str.lower().str.replace('[^\w\s,]', '')
movie_reviews['originalLanguage'] = movie_reviews['originalLanguage'].str.lower().str.replace('[^\w\s,]', '')
#preprocessing averageScore column
movie_reviews['averageScore'] = movie_reviews['averageScore'].astype(float)


# Feature extraction for genre
mlb = MultiLabelBinarizer()
genres = mlb.fit_transform(movie_reviews['genre'])


# Feature extraction for director
director_encoder = LabelEncoder()
directors = director_encoder.fit_transform(movie_reviews['director'])
# Feature extraction for original language
language_encoder = OneHotEncoder(max_categories=10, sparse_output=True, handle_unknown='ignore')
language=language_encoder.fit_transform(movie_reviews['originalLanguage'].values.reshape(-1, 1))

# Feature extraction for averageScore
mms = MinMaxScaler()
avg_scores = mms.fit_transform(movie_reviews['averageScore'].values.reshape(-1, 1))
#combining all features into a single dataframe





In [27]:
print(matrix.dtype)
print(avg_scores.dtype)
print(language.dtype)
print(genres.dtype)
print(directors.dtype)

float64
float64
float64
int32
int32


##### ***Combining all features into a single dataframe, we are using concatenate function an d we are only combining the genre, director, language and averageScore those are dense features***

In [26]:
#!!important!!
# Combine all features
combined_features = np.concatenate((genres, directors.reshape(-1, 1), language.toarray(), avg_scores,),axis=1)

In [27]:
from sklearn.decomposition import TruncatedSVD

###### ***We are using TruncatedSVD to reduce the dimension of the reviewText, we are using 1000 components for reducing the dimension to 1000 that is why it will take some time ,leave it for few minutes to complete***

In [28]:
#!!important!!
# Apply SVD to the reviewText feature

svd = TruncatedSVD(n_components=1000)
reviews =svd.fit_transform(matrix)


###### ***Combining the combined_features and reviews into a single dataframe i.e, dense matrix and sparse matrix together , we are using hstack function***

In [29]:
#!!important!!
#combining all features

complete_features= np.hstack((combined_features, reviews))

In [30]:
#!!important!!
#reset the index of the dataframe
#making a indices to search movie using title


df_movie = df_movie.reset_index()
indices = pd.Series(df_movie.index, index=df_movie['title'])

In [41]:
print(complete_features)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  2.55373247e-03
   2.29568463e-03  2.30525628e-02]
 [ 1.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 1.00000000e+00  0.00000000e+00  0.00000000e+00 ... -8.14189202e-03
   9.19269570e-04 -1.60404680e-02]
 [ 1.00000000e+00  1.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.90548691e-02
  -1.71107345e-02 -1.56152137e-02]]


In [31]:
#!!important!!
#ormalize the features
normalized_features = normalize(complete_features, axis=0)
# num_features =normalized_features.shape[1]

# Assign weights to each feature
# weights = np.array([1.0, 0.6, 0.5,  0.8, 0.7])
# assert len(weights) == num_features, "Weights length does not match the number of features"

# Apply weights to the normalized features
# weighted_features = normalized_features * weights[:, np.newaxis]

#

In [38]:
Import Levenshtein
#assigning weights to the features
# weights = np.array([0.9,0.6,0.4,0.8,0.7])

# #calculating the weighted sum of the features
# weighted_sum = np.dot(normalized_features, weights)
# #create a function to handle recommendation process
# user_movie = entry_movie.get()

#     # Find the closest matching movie title using Levenshtein distance
# closest_match = min(df_movie['title'], key=lambda x: Levenshtein.distance(user_movie.lower(), df_movie['title'].str.lower()))

# if Levenshtein.distance(user_movie.lower(), closest_match.lower()) > 2:  # Set a threshold for the distance
#     messagebox.showerror("Error", "Movie not found!")
#     return

#     closest_movie = closest_match
#     messagebox.showinfo("Closest Match", f"Closest matching movie: {closest_movie}")

#     # Find the index of the closest matching movie in the 'movies' list
#     user_movie_index = indices(closest_movie)

#     # Apply weights to the normalized features
#     weighted_features = normalized_features * np.array(weights)

#     # Calculate cosine similarity between the user input movie and all other movies
#     cosine_similarities = cosine_similarity(weighted_features[user_movie_index].reshape(1, -1), weighted_features)

#     # Sort the movies based on cosine similarity scores
#     similar_movies_indices = np.argsort(cosine_similarities).flatten()[::-1]  # Descending order

#     # Recommend top 10 movies
#     top_10_movies = [movies[idx] for idx in similar_movies_indices[:10]]

#     # Show the recommended movies in a message box
#     messagebox.showinfo("Recommended Movies", "\n".join(top_10_movies))

# # Create the GUI
# window = tk.Tk()
# window.title("Movie Recommendation System")

# # Create a label and entry field for the movie input
# label_movie = tk.Label(window, text="Enter a movie:")
# label_movie.pack()
# entry_movie = tk.Entry(window)
# entry_movie.pack()

# # Create a button to trigger the recommendation process
# button_recommend = tk.Button(window, text="Recommend", command=recommend_movies)
# button_recommend.pack()

# # Run the GUI
# window.mainloop()



ModuleNotFoundError: No module named 'levenshtein'

###### ***Here we are assigning weights to the features and applying cosine similarity to fin d the most similar movies***

In [31]:
weights = [0.9, 0.5, 0.3, 0.8,0.7]  # Example weights for each feature

# Normalize the combined features


# User input movie
user_movie = "Vanilla Sky"  # Example user input movie

# Find the index of the user input movie in the 'movies' list

user_movie_index = indices[user_movie]

#normalize  weights to the features
# weighted_features = normalized_features * np.array(weights)



# # Calculate cosine similarity between the user input movie and all other movies
# cosine_similarities = cosine_similarity(weighted_features[user_movie_index].reshape(1, -1), weighted_features)
#experiment

n_features=normalized_features.shape[1]
featured_weights=weights*(n_features // 5)
remaining_features = n_features % 5
featured_weights += [0.1] * remaining_features
weights_matrix = np.diag(featured_weights)


# user_weighted_features = normalized_features[user_movie_index].reshape(-1, 1)  # Reshape to (number_of_features, 1)
weighted_features = normalized_features @ weights_matrix  # Multiplication with broadcasting
cosine_similarities = cosine_similarity(weighted_features[user_movie_index].reshape(1, -1), weighted_features)


# Sort the movies based on cosine similarity scores
similar_movies_indices = np.argsort(cosine_similarities).flatten()[::-1]  # Descending order

# Recommend top 10 movies
top_10_movies = [df_movie['title'].iloc[i] for i in similar_movies_indices[1:11]]

# Print the recommended movies
print("Recommended movies:")
for movie in top_10_movies:
    print(movie)


Recommended movies:
Aloha
Elizabethtown
We Bought a Zoo
Jerry Maguire
Singles
The Water Diviner
Almost Famous
Unhinged
A Good Year
A Beautiful Mind


###### *an interface to get the input from the user and display the recommended movies*

*We are using Tkinter to create a user interface to get the input from the user and display the recommended movies. and we are using the Levenshtein distance to find the closest matching movie title.* 

In [33]:
weights = [0.9, 0.5, 0.3, 0.8,0.7]  # Example weights for each feature
n_features=normalized_features.shape[1]
featured_weights=weights*(n_features // 5)
remaining_features = n_features % 5
featured_weights += [0.1] * remaining_features
weights_matrix = np.diag(featured_weights)

weighted_features = normalized_features @ weights_matrix  # Multiplication with broadcasting
def recommend_movies():
    # Get the user input movie
    user_movie = entry_movie.get()

    # Find the closest matching movie title using Levenshtein distance
    closest_match = min(df_movie['title'], key=lambda x: Levenshtein.distance(user_movie.lower(), x.lower()))

    if Levenshtein.distance(user_movie.lower(), closest_match.lower()) > 2:  # Set a threshold for the distance
        messagebox.showerror("Error", "Movie not found!")
        return

    closest_movie = closest_match
    messagebox.showinfo("Closest Match", f"Closest matching movie: {closest_movie}")

    # Find the index of the closest matching movie in the 'movies' list
    user_movie_index = indices[closest_movie]

    # Apply weights to the normalized features
    

    # Calculate cosine similarity between the user input movie and all other movies
    cosine_similarities = cosine_similarity(weighted_features[user_movie_index].reshape(1, -1), weighted_features)

    # Sort the movies based on cosine similarity scores
    similar_movies_indices = np.argsort(cosine_similarities).flatten()[::-1]  # Descending order

    # Recommend top 10 movies
    top_10_movies = [df_movie['title'].iloc[i] for i in similar_movies_indices[1:11]]

    # Show the recommended movies in a message box
    messagebox.showinfo("Recommended Movies", "\n".join(top_10_movies))

# Create the GUI
window = tk.Tk()
window.title("Movie Recommendation System")

# Create a label and entry field for the movie input
label_movie = tk.Label(window, text="Enter a movie:")
label_movie.pack()
entry_movie = tk.Entry(window)
entry_movie.pack()

# Create a button to trigger the recommendation process
button_recommend = tk.Button(window, text="Recommend", command=recommend_movies)
button_recommend.pack()

# Run the GUI
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\gaura\AppData\Local\Temp\ipykernel_8228\1197692713.py", line 30, in recommend_movies
    cosine_similarities = cosine_similarity(weighted_features[user_movie_index].reshape(1, -1), weighted_features)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\sklearn\metrics\pairwise.py", line 1657, in cosine_similarity
    X, Y = check_pairwise_arrays(X, Y)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\sklearn\metrics\pairwise.py", line 189, in check_pairwise_arrays
    raise ValueError(
ValueErro